In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing

In [5]:
files=['../input/sejong-ai-challenge-p3/sample_submission.csv',
       '../input/sejong-ai-challenge-p3/test.csv',
       '../input/sejong-ai-challenge-p3/train.csv']

with multiprocessing.Pool() as pool:
    submission,test,train=pool.map(pd.read_csv,files)

In [6]:
def datainfo(df):
    return pd.DataFrame([(col,df[col].unique()[:5],df[col].nunique(),df[col].isna().sum(),
                         df[col].dtype) for col in df.columns],
                       columns=['name','value :5','nunique','missing','dtype'])

datainfo(train)

,name,value :5,nunique,missing,dtype
0,id,"[1, 3, 4, 5, 6]",647,0,int64
1,y,"[5.7784379470000005, 4.085856671, 5.615403883,...",647,0,float64
2,bw,"[1559, 1000, 1430, 1984, 1320]",208,0,int64
3,b.head,"[28.64852128, 25.0, 29.0, 31.0, 28.0]",45,0,float64
4,preterm,"[10, 8, 6, 2, 7]",14,0,int64
5,birth.o,"[2, 4, 1, 3]",4,0,int64
6,nnhealth,"[94, 89, 112, 99, 110]",83,0,int64
7,momage,"[33, 22, 20, 23, 37]",30,0,int64
8,dadage,"[nan, 31.0, 22.0, 27.0, 26.0]",13,525,float64
9,sex,"[1, 0]",2,0,int64


In [7]:
# col=[col for col in train.columns if not col in ['id','dadage']]

# sns.pairplot(train[col])

In [8]:
idx=train[train['momage']==43].index
train=train.drop(idx,axis=0)

In [9]:
a=train['mom.edu'].median()
train['mom.edu']=train['mom.edu'].fillna(a)

b=test['mom.edu'].median()
test['mom.edu']=test['mom.edu'].fillna(b)

In [10]:
# array=train[train['mom.edu'].isna()]['momage'].unique()
# for i in array:
#     a=train[train['momage']==i]['mom.edu'].mean()      
#     train.loc[train['momage']==i,'mom.edu']=train.loc[train['momage']==i,'mom.edu'].fillna(float(a))
# train['mom.edu']=train['mom.edu'].apply(lambda x:round(x))

# array1=test[test['mom.edu'].isna()]['momage'].unique()
# for i in array1:
#     b=test[test['momage']==i]['mom.edu'].mean()      
#     test.loc[test['momage']==i,'mom.edu']=test.loc[test['momage']==i,'mom.edu'].fillna(float(a))
# test['mom.edu']=test['mom.edu'].apply(lambda x:round(x))

In [11]:
a=train['site'].median()
train['site']=train['site'].fillna(a)
train.drop('dadage',axis=1,inplace=True)

b=test['site'].median()
test['site']=test['site'].fillna(a)
test.drop('dadage',axis=1,inplace=True)

In [12]:
# cols=[col for col in train.columns if not col in ['id']]

# for col in cols:
#     sns.boxplot(train[col])
#     plt.title(col)
#     plt.figure(figsize=(4,3))
#     plt.show()

In [13]:
from sklearn.preprocessing import MinMaxScaler

cols=['b.head','birth.o','nnhealth']

scaler = MinMaxScaler()

for col in cols:
    scaler.fit(train[col].values.reshape(-1,1))
    train[col]=scaler.transform(train[col].values.reshape(-1,1))

for col in cols:
    scaler.fit(test[col].values.reshape(-1,1))
    test[col]=scaler.transform(test[col].values.reshape(-1,1))

In [14]:
# cols=[col for col in test.columns if not col in ['id']]

# for col in cols:
#     sns.boxplot(test[col])
#     plt.title(col)
#     plt.figure(figsize=(4,3))
#     plt.show()

In [15]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

models=[]

X=train.drop(['id','y'],axis=1)
y=train['y']
X_test=test.drop('id',axis=1)

skf=KFold(n_splits=5,shuffle=True,random_state=2020)

for train_idx,valid_idx in skf.split(X):
    X_tr,X_val=X.iloc[train_idx],X.iloc[valid_idx]
    y_tr,y_val=y.iloc[train_idx],y.iloc[valid_idx]
    
    model = Ridge(alpha=7,solver='cholesky',random_state=2020)
    model.fit(X_tr,y_tr)
    models.append(model)
    pred=model.predict(X_val)
    
    score=mean_squared_error(pred,y_val)
    print(score)
    

4.026665978795691
7.568716771670022
4.465400795044007
8.605764323657885
6.150545803806579


In [16]:
# from sklearn.model_selection import KFold
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.metrics import mean_squared_error

# models=[]

# X=train.drop(['id','y'],axis=1)
# y=train['y']
# X_test=test.drop('id',axis=1)

# skf=KFold(n_splits=5,shuffle=True,random_state=2020)

# for train_idx,valid_idx in skf.split(X,y):
#     X_tr,X_val=X.iloc[train_idx],X.iloc[valid_idx]
#     y_tr,y_val=y.iloc[train_idx],y.iloc[valid_idx]
    
#     model = GradientBoostingRegressor(learning_rate=0.05,n_estimators=400,
#                                   subsample=0.8,max_depth=5,random_state=2020)
#     model.fit(X_tr,y_tr)
#     models.append(model)
#     pred=model.predict(X_val)
    
#     score=mean_squared_error(pred,y_val)
#     print(score)
    

In [17]:
# from sklearn.model_selection import GridSearchCV

# parameters = {'alpha':[1,5,9,15], 'solver':['svd','auto','cholesky','lsqr',
#                                          'sparse_cg','sag']}

# clf = GridSearchCV(models[0], parameters)
# clf.fit(, iris.target)
# pred=clf.predict(X_test)

NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
pred

In [ ]:
submission['y']=models[0].predict(X_test)

In [ ]:
submission

In [ ]:
submission.to_csv('17010668(2).csv',index=False)